## Load Script

### The raw data file littered with invalid escape characters and complex combination of colons. The `json` file had to be first loaded as raw text and converted into valid `json` format before loading as `json` file.

In [1]:
import json
import pandas as pd
import fastparquet as fpq

def clean_escape_chars(text: str):
    return (
        text.replace(r"\\'", "'")
        
        .replace(r"\\\t", " ")
        .replace(r"\\t", " ")
        .replace(r"\t", " ")

        .replace(r"\'", "'")
        .replace(r"\\x", " ")
        .replace(r"\x", " ")
        .replace(r"\\u", " ")
        .replace(r"\\\\", " ")
    )

def load_data(file_name):
    with open(f"./{file_name}.json", "r", encoding="utf-8") as f:
        data = f.readlines()

    print(f"Loaded {len(data)} strings.")

    s_list = []
    for i in data:
        left, mid, right = i.partition("'review/text': ")
        if left == "" or mid == "" or right == "":
            continue
        mid = '"review/text": '

        left = left.replace('"', "'")
        left = left.replace("{'", '{"')
        left = left.replace("', '", '", "')
        left = left.replace("': '", '": "')
        left = left.replace("', ", '", ')
        left = clean_escape_chars(left)

        right = right.replace('"', "'")
        right = right.replace("'}\n", '"}')
        right = right.replace("'", '"', 1)
        right = clean_escape_chars(right)

        s_list.append(" ".join([left, mid, right]))

    dict_data = [json.loads(i) for i in s_list]

    print(f"Converted {len(dict_data)} dictionaries")

    del s_list
    del data

    df = pd.DataFrame(dict_data)
    fpq.write(f'{file_name}.parquet', df)
    print(f"Wrote {file_name}.parquet")

file_names = ("beeradvocate","ratebeer")

for file in file_names:
    print(f"=== Loading {file} ===")
    load_data(file)
    print("="*30)


=== Loading beeradvocate ===
Loaded 1586615 strings.
Converted 1586614 dictionaries
Wrote beeradvocate.parquet
=== Loading ratebeer ===
Loaded 2924164 strings.
Converted 2924163 dictionaries
Wrote ratebeer.parquet
